In [1]:
# import os
# import pandas as pd
# import pyreadstat
# import yaml
# import numpy as np
# from scipy.stats import wasserstein_distance
# from sklearn.model_selection import train_test_split
# from tqdm import tqdm
# import pingouin as pg  # For Cronbach's alpha
# import warnings
# import glob
# warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# # Configuration
# USE_SELECTED_QUESTIONS_ONLY = True  # Set to False to use all questions from the survey

# # Base directories
# # base_dir = '/Users/vavre/Research/TeIAS/impersonation'
# base_dir = '/home/erfan/Desktop/actives/bsop'
# # /home/erfan/Desktop/actives/bsop/anthology/data/ATPs/W34_Apr18/ATP W34.sav
# questions_dir = f'{base_dir}/anthology/data/questions/data/questions'
# output_dir = f'{base_dir}/results'

# # Create output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# # Hard-coded paths to survey files
# survey_files = {
#     # '37': f'{base_dir}/ATP/W37_Jul18/ATP W37.sav',
#     '34': f'{base_dir}/anthology/data/ATPs/W34_Apr18/ATP W34.sav',
#     '99': f'{base_dir}/anthology/data/ATPs/W99_Nov21/ATP W99.sav',
#     '92': f'{base_dir}/anthology/data/ATPs/W92_Jul21/ATP W92.sav',
#     # Add more waves as needed
# }


# print(f"Processing {len(survey_files)} survey waves: {', '.join(sorted(survey_files.keys()))}")

# def process_wave(wave, survey_path, use_selected_questions):
#     print(f"\n{'='*80}\nProcessing Wave {wave}\n{'='*80}")
    
#     # Try to load the survey data first to check if it exists
#     try:
#         df, meta = pyreadstat.read_sav(survey_path)
#         print(f"Loaded survey data with {len(df)} responses and {len(df.columns)} variables")
#     except Exception as e:
#         print(f"Error reading survey file {survey_path}: {e}")
#         return None
    
#     # Find the questions to use
#     use_all_questions = False  # Local flag for this wave
    
#     if use_selected_questions:
#         # Look for corresponding questions file
#         questions_path = f'{questions_dir}/ATP_W{wave}_demographic_questions.yaml'
#         if not os.path.exists(questions_path):
#             print(f"No demographic question file found at {questions_path}")
#             # Look for any question file for this wave
#             alt_questions_paths = glob.glob(f'{questions_dir}/ATP_W{wave}_*.yaml')
#             if alt_questions_paths:
#                 questions_path = alt_questions_paths[0]
#                 print(f"Using alternative question file: {os.path.basename(questions_path)}")
#             else:
#                 print(f"No question files found for wave {wave}. Will use all questions.")
#                 use_all_questions = True
        
#         # Load selected questions if a file was found
#         if not use_all_questions:
#             try:
#                 with open(questions_path, 'r') as file:
#                     selected_questions = list(yaml.safe_load(file)['questions'].keys())
#                 print(f"Loaded {len(selected_questions)} questions from YAML file")
#             except Exception as e:
#                 print(f"Error loading questions from {questions_path}: {e}")
#                 print("Will use all questions instead")
#                 use_all_questions = True
#     else:
#         # Global flag set to use all questions
#         use_all_questions = True
    
#     # If using all questions
#     if use_all_questions:
#         # Common metadata column prefixes/suffixes to exclude
#         exclude_patterns = [
#             'WEIGHT', 'ID', 'CASEID', 'PSU', 'STRATUM', 'QKEY', 'DATE', 
#             'START', 'END', 'DURATION', 'MODE', 'VERSION', 'LANGUAGE',
#             'FLAG', 'CONSENT', 'COMPLETE', 'STATUS'
#         ]
        
#         # Filter out metadata columns
#         selected_questions = []
#         for col in df.columns:
#             # Skip if column name contains any excluded pattern
#             if any(pattern in col.upper() for pattern in exclude_patterns):
#                 continue
#             # Only include columns with numeric data (for survey responses)
#             if pd.api.types.is_numeric_dtype(df[col]):
#                 selected_questions.append(col)
        
#         print(f"Using {len(selected_questions)} questions from all available survey variables")
    
#     # Ensure we have the correct weight column
#     weight_column = f'WEIGHT_W{wave}'
#     if weight_column not in df.columns:
#         print(f"Warning: Weight column {weight_column} not found. Available columns that may contain weights:")
#         weight_cols = [col for col in df.columns if 'WEIGHT' in col]
#         print(weight_cols)
#         if weight_cols:
#             weight_column = weight_cols[0]
#             print(f"Using {weight_column} instead.")
#         else:
#             print("No weight column found. Proceeding without weights.")
#             weight_column = None

#     print(f"Using weight column: {weight_column}")

#     # Function to calculate Frobenius norm between two distribution matrices
#     def frobenius_norm(dist1, dist2):
#         # Create a common index of all unique values
#         all_values = sorted(set(dist1.index) | set(dist2.index))
        
#         # Create arrays with values for each distribution
#         dist1_array = np.array([dist1.get(val, 0) for val in all_values])
#         dist2_array = np.array([dist2.get(val, 0) for val in all_values])
        
#         # Calculate Frobenius norm
#         return np.sqrt(np.sum((dist1_array - dist2_array) ** 2))

#     # Initialize dictionaries to store results
#     question_metrics = {q: {'wd': [], 'frobenius': [], 'cronbach_alpha': []} for q in selected_questions}
#     aggregated_metrics = {'wd': {}, 'frobenius': {}, 'cronbach_alpha': {}}

#     # Clean non-compliant responses
#     df_clean = df.copy()
#     for q in selected_questions:
#         if q in df_clean.columns:
#             # Replace values < 0 (usually error codes)
#             df_clean.loc[df_clean[q] < 0, q] = np.nan
#             # Replace values > 90 (usually refused/don't know codes in ATP surveys)
#             df_clean.loc[df_clean[q] > 90, q] = np.nan

#     # Run 100 iterations of random splits
#     num_iterations = 100
#     for iteration in tqdm(range(num_iterations), desc=f"Processing Wave {wave}"):
#         # For each question, calculate metrics between two random splits
#         for q in selected_questions:
#             try:
#                 # Skip if question not in dataframe
#                 if q not in df_clean.columns:
#                     continue
                    
#                 responses = df_clean[q]
                
#                 # Skip if too many missing values
#                 if responses.isna().mean() > 0.5:
#                     continue
                
#                 # Get the weights (if available)
#                 weights = df_clean[weight_column] if weight_column else pd.Series(1, index=df_clean.index)
                
#                 # Split the indices into two random groups
#                 indices = responses.dropna().index
#                 if len(indices) < 10:  # Skip questions with very few responses
#                     continue
                    
#                 # Split indices and get corresponding responses and weights
#                 train_indices, test_indices = train_test_split(
#                     indices, 
#                     test_size=0.5,
#                     random_state=None,  # Different splits each iteration
#                     shuffle=True
#                 )
                
#                 r1 = responses.loc[train_indices]
#                 r2 = responses.loc[test_indices]
#                 w1 = weights.loc[train_indices]
#                 w2 = weights.loc[test_indices]
                
#                 # Calculate weighted distributions
#                 # First, get unique response values (excluding NaN)
#                 unique_vals = pd.Series(list(set(r1.dropna()) | set(r2.dropna())))
                
#                 # Create weighted distributions
#                 r1_dist = {}
#                 r2_dist = {}
                
#                 for val in unique_vals:
#                     # Sum weights for each response value
#                     r1_mask = (r1 == val)
#                     r2_mask = (r2 == val)
                    
#                     if r1_mask.any():
#                         r1_dist[val] = w1[r1_mask].sum()
#                     else:
#                         r1_dist[val] = 0
                        
#                     if r2_mask.any():
#                         r2_dist[val] = w2[r2_mask].sum()
#                     else:
#                         r2_dist[val] = 0
                
#                 # Convert to pandas Series
#                 r1_dist = pd.Series(r1_dist)
#                 r2_dist = pd.Series(r2_dist)
                
#                 # Normalize to get probability distributions
#                 if r1_dist.sum() > 0:
#                     r1_dist = r1_dist / r1_dist.sum()
#                 if r2_dist.sum() > 0:
#                     r2_dist = r2_dist / r2_dist.sum()
                
#                 # Check if values are numeric for Wasserstein distance
#                 all_values = sorted(set(r1_dist.index) | set(r2_dist.index))
#                 numeric_values = []
#                 is_numeric = True
                
#                 for x in all_values:
#                     if pd.isna(x):
#                         continue
#                     elif isinstance(x, (int, float)):
#                         numeric_values.append(x)
#                     else:
#                         try:
#                             # Try converting to numeric
#                             num_val = float(x)
#                             numeric_values.append(num_val)
#                         except:
#                             is_numeric = False
#                             break
                
#                 # Calculate Wasserstein Distance if values are numeric
#                 if is_numeric and len(numeric_values) > 1:
#                     try:
#                         # Get probability arrays for numeric values
#                         p1 = np.array([r1_dist.get(val, 0) for val in numeric_values])
#                         p2 = np.array([r2_dist.get(val, 0) for val in numeric_values])
                        
#                         # Normalize if needed
#                         if p1.sum() > 0:
#                             p1 = p1 / p1.sum()
#                         if p2.sum() > 0:
#                             p2 = p2 / p2.sum()
                        
#                         wd = wasserstein_distance(numeric_values, numeric_values, p1, p2)
#                         question_metrics[q]['wd'].append(wd)
#                     except Exception as e:
#                         # Skip silently
#                         pass
                
#                 # Calculate Frobenius Norm
#                 try:
#                     frobenius = frobenius_norm(r1_dist, r2_dist)
#                     question_metrics[q]['frobenius'].append(frobenius)
#                 except Exception as e:
#                     # Skip silently
#                     pass
                
#                 # Prepare data for Cronbach's Alpha
#                 try:
#                     # For Cronbach's Alpha, we need original responses, not distributions
#                     # Create new indices to avoid duplicates
#                     r1_reset = r1.reset_index(drop=True)
#                     r2_reset = r2.reset_index(drop=True)
                    
#                     # Create a dataframe with responses and group indicators
#                     df_alpha = pd.DataFrame({
#                         'response': pd.concat([r1_reset, r2_reset]),
#                         'group': pd.concat([pd.Series(['g1']*len(r1_reset)), pd.Series(['g2']*len(r2_reset))])
#                     })
                    
#                     # Only proceed if responses are numeric
#                     if pd.to_numeric(df_alpha['response'], errors='coerce').notna().all():
#                         # Create pivot table for Cronbach's alpha calculation
#                         pivot_data = pd.pivot_table(
#                             df_alpha, 
#                             values='response', 
#                             index=df_alpha.index, 
#                             columns='group', 
#                             aggfunc='first'
#                         )
                        
#                         # Calculate Cronbach's Alpha
#                         if pivot_data.shape[1] >= 2:  # Need at least 2 columns
#                             alpha_result = pg.cronbach_alpha(pivot_data)
#                             alpha = alpha_result[0]  # Extract only the alpha value
#                             question_metrics[q]['cronbach_alpha'].append(alpha)
#                 except Exception as e:
#                     # Skip if Cronbach's alpha calculation fails
#                     pass
            
#             except Exception as e:
#                 # Skip silently
#                 continue

#     # Calculate average metrics for each question
#     for q in selected_questions:
#         if question_metrics[q]['wd']:
#             aggregated_metrics['wd'][q] = np.mean(question_metrics[q]['wd'])
        
#         if question_metrics[q]['frobenius']:
#             aggregated_metrics['frobenius'][q] = np.mean(question_metrics[q]['frobenius'])
        
#         if question_metrics[q]['cronbach_alpha']:
#             aggregated_metrics['cronbach_alpha'][q] = np.mean(question_metrics[q]['cronbach_alpha'])

#     # Print results
#     print(f"\nHuman Baseline Metrics for Wave {wave}:")
#     print("-" * 80)
#     print("Average Wasserstein Distance across all questions:", 
#           np.mean(list(aggregated_metrics['wd'].values())) if aggregated_metrics['wd'] else "N/A")
#     print("Average Frobenius Norm across all questions:", 
#           np.mean(list(aggregated_metrics['frobenius'].values())) if aggregated_metrics['frobenius'] else "N/A")
#     print("Average Cronbach's Alpha across all questions:", 
#           np.mean(list(aggregated_metrics['cronbach_alpha'].values())) if aggregated_metrics['cronbach_alpha'] else "N/A")
#     print("-" * 80)

#     # Calculate how many questions had valid metrics
#     valid_wd = len(aggregated_metrics['wd'])
#     valid_fro = len(aggregated_metrics['frobenius'])
#     valid_alpha = len(aggregated_metrics['cronbach_alpha'])
#     print(f"Questions with valid metrics (out of {len(selected_questions)}):")
#     print(f"  Wasserstein Distance: {valid_wd} ({valid_wd/len(selected_questions)*100:.1f}%)")
#     print(f"  Frobenius Norm: {valid_fro} ({valid_fro/len(selected_questions)*100:.1f}%)")
#     print(f"  Cronbach's Alpha: {valid_alpha} ({valid_alpha/len(selected_questions)*100:.1f}%)")

#     # Prepare results dataframe
#     results_df = pd.DataFrame({
#         'question': selected_questions,
#         'wasserstein_distance': [aggregated_metrics['wd'].get(q, np.nan) for q in selected_questions],
#         'frobenius_norm': [aggregated_metrics['frobenius'].get(q, np.nan) for q in selected_questions],
#         'cronbach_alpha': [aggregated_metrics['cronbach_alpha'].get(q, np.nan) for q in selected_questions]
#     })

#     # Add a row with overall averages
#     overall_row = pd.DataFrame({
#         'question': ['OVERALL_AVERAGE'],
#         'wasserstein_distance': [np.mean(list(aggregated_metrics['wd'].values())) if aggregated_metrics['wd'] else np.nan],
#         'frobenius_norm': [np.mean(list(aggregated_metrics['frobenius'].values())) if aggregated_metrics['frobenius'] else np.nan],
#         'cronbach_alpha': [np.mean(list(aggregated_metrics['cronbach_alpha'].values())) if aggregated_metrics['cronbach_alpha'] else np.nan]
#     })

#     # Append the overall row
#     results_df = pd.concat([results_df, overall_row], ignore_index=True)

#     # Create output filename with flag indicator
#     question_source = "selected" if not use_all_questions else "all"
#     output_path = f'{output_dir}/human_baseline_weighted_W{wave}_{question_source}_questions.csv'
#     results_df.to_csv(output_path, index=False)
#     print(f"Results saved to: {output_path}")
    
#     return {
#         'wave': wave,
#         'question_source': question_source,
#         'num_questions': len(selected_questions),
#         'valid_wd': valid_wd,
#         'valid_frobenius': valid_fro,
#         'valid_alpha': valid_alpha,
#         'avg_wd': np.mean(list(aggregated_metrics['wd'].values())) if aggregated_metrics['wd'] else np.nan,
#         'avg_frobenius': np.mean(list(aggregated_metrics['frobenius'].values())) if aggregated_metrics['frobenius'] else np.nan,
#         'avg_alpha': np.mean(list(aggregated_metrics['cronbach_alpha'].values())) if aggregated_metrics['cronbach_alpha'] else np.nan
#     }


# # Process all waves and collect summary results
# summary_results = []
# for wave, survey_path in survey_files.items():
#     result = process_wave(wave, survey_path, USE_SELECTED_QUESTIONS_ONLY)
#     if result:
#         summary_results.append(result)

# # Create and save summary dataframe
# if summary_results:
#     question_source = "selected" if USE_SELECTED_QUESTIONS_ONLY else "all"
#     summary_df = pd.DataFrame(summary_results)
#     summary_output_path = f'{output_dir}/human_baseline_summary_{question_source}_questions.csv'
#     summary_df.to_csv(summary_output_path, index=False)
    
#     print(f"\n{'='*80}\nSummary of All Waves\n{'='*80}")
#     print(summary_df)
#     print(f"\nSummary saved to: {summary_output_path}")
# else:
#     print("\nNo results were generated for any wave.")




In [2]:
import pandas as pd
import numpy as np
import pyreadstat, json, yaml

from scipy.stats import wasserstein_distance
from pingouin import cronbach_alpha

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pprint import pprint
import random

from minilib import *

In [10]:
survey_paths = [
    '../data/ATPs/W92_Jul21/ATP W92.sav',
    '../data/ATPs/W34_Apr18/ATP W34.sav',
    '../data/ATPs/W99_Nov21/ATP W99.sav',
]


for survey_path in survey_paths:
    print(40*'=')
    wave = survey_path.split()[-1].split('.')[0][1:]
    weight_col = f'WEIGHT_W{wave}'
    print(f'{wave = }')

    selected_questions_path = f'/home/erfan/Desktop/actives/bsop/anthology/configs/surveys/questionnaire/ATP/ATP_W{wave}_questions.yaml'

    # [STEP] get_human_data()
    df, meta = pyreadstat.read_sav(survey_path)

    # with open(selected_questions_path, 'r') as file:
    #     selected_questions = json.load(file)
        
    with open(selected_questions_path, 'r') as file:
        selected_questions = list(yaml.safe_load(file)['questions'].keys())
        
        
    # selected_questions = [col for col in meta.column_names if col.endswith(f'_W{wave}') and col not in [f'INTERVIEW_START_W{wave}', f'INTERVIEW_END_W{wave}', f'DEVICE_TYPE_W{wave}', f'LANG_W{wave}', f'FORM_W{wave}']]
    print(f'{len(selected_questions) = }')
    print(f'{len(df) = }')
    
    df = remove_non_compliant_responses(df, selected_questions)

    # for _ in tqdm(range(100)):
    df_comp = df.sample(2500, replace=False)
    wd = distance_lower_bound(df_comp, selected_questions, weight_column=weight_col, dist_type='EMD', num_iter=100)['average EMD'].item()
    fro = matrix_dist_lower_bound(df_comp, selected_questions, wave=wave, dist_type="Frobenius", num_iter=100).item()
    ca = cronbach_alpha(df_comp[selected_questions])[0].item()
        
    print(f'WD: {round(wd, 3)}')
    print(f'Fro: {round(fro, 3)}')
    print(f'α: {round(ca, 3)}')


wave = '92'
len(selected_questions) = 7
len(df) = 10916
WD: 0.083
Fro: nan
α: 0.54
wave = '34'
len(selected_questions) = 8
len(df) = 2537
WD: 0.05
Fro: 0.425
α: 0.785
wave = '99'
len(selected_questions) = 6
len(df) = 10260
WD: 0.09
Fro: 0.313
α: 0.829


{'POSNEGAI_a_W99': 0.08919909328120144,
 'POSNEGAI_b_W99': 0.07687652650343586,
 'POSNEGAI_c_W99': 0.08928317061439697,
 'POSNEGAI_d_W99': 0.09305834441006074,
 'POSNEGAI_e_W99': 0.084745965747097,
 'POSNEGAI_f_W99': 0.09304918205766328,
 'average EMD': np.float64(0.08770204710230922)}

In [5]:
df_comp_X = df_comp[selected_questions].values
df_comp_w = df_comp[weight_col].values

_, human_weighted_cov = get_weighted_cov(df_comp_X, df_comp_w, corr=True)

In [6]:
cov_matrix_distance(
                human_weighted_cov,
                df_comp[selected_questions].corr().values,
                dist_type="Frobenius",
            )

np.float64(0.14536221536353883)

In [7]:
# df = remove_non_compliant_responses(df, selected_questions)

# qwd = {k: [] for k in selected_questions}

# for _ in tqdm(range(100)):
#     df_comp = df.sample(2500, replace=False)
#     i1, i2 = train_test_split(
#         df_comp.index,
#         test_size=0.5,
#         random_state=None, # Use None for different splits each iteration
#         shuffle=True
#     )
        
#     qd1, _ = get_human_response(df_comp.loc[i1], selected_questions, df_comp[f'WEIGHT_W{wave}'].loc[i1])
#     qd2, _ = get_human_response(df_comp.loc[i2], selected_questions, df_comp[f'WEIGHT_W{wave}'].loc[i2])
    
#     for q in selected_questions:
#         d1, d2 = qd1[q], qd2[q]
#         assert len(d1) == len(d2), 'Dists have different likerts'
#         unique_responses = sorted(list(df_comp[q].value_counts(normalize=True).index))
#         assert len(unique_responses) == len(d1), 'Dists miss likerts'
        
#         wd = wasserstein_distance(unique_responses, unique_responses, d1, d2)
#         qwd[q].append(wd.item())